In [1]:
#Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping

### Data Loading

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def getSampleData():
  return pd.read_csv("/content/drive/MyDrive/Sample getTransactions API data.csv")

In [4]:
raw = getSampleData()
raw.head(30)

,type,id,status,description,amount,account,balance,direction,class,institution,connection,enrich,transactionDate,postDate,subClass,links
0,transaction,4957bcf6-f18f-43d7-94df-71f44a0bcf32,posted,Wdl ATM WES IGA BALGOWLAH HGT BALGOWL AU,-200.0,d3de1ca1,22109.56,debit,cash-withdrawal,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:07:36,NaN,"{""account"":""https://au-api.basiq.io/users/6a52..."
1,transaction,0d6e11ab-e28e-4de0-a152-600cc44fb61c,posted,Non Hooli ATM Withdrawal Fee,-2.5,070c1d68-0ee0-477a-9679-294ea7059939,22309.56,debit,bank-fee,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,"{\title\"":\""\""","\""code\"":\""card\""}"""
2,transaction,323f6d42-a38e-4c46-83d2-6c8f3e999686,posted,Wdl ATM WES IGA BALGOWLAH HGT BALGOWL AU,-200.0,070c1d68-0ee0-477a-9679-294ea7059939,22312.06,debit,cash-withdrawal,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,NaN,"{\self\"":\""https://au-api.basiq.io/users/bca4b..."
3,transaction,1ab3a3c5-faeb-4de3-b5aa-612e5bc76fd5,posted,Non Hooli ATM Withdrawal Fee,-2.5,070c1d68-0ee0-477a-9679-294ea7059939,22512.06,debit,bank-fee,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,3/08/2023,2023-08-03T00:00:00Z,"{\title\"":\""\""","\""code\"":\""card\""}"""
4,transaction,d12df6a2-48b6-4f5f-a718-ccb377aa330c,posted,Transfer Platnm Homeloan 346454,-3852.5,070c1d68-0ee0-477a-9679-294ea7059939,22514.56,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,29/07/2023,2023-07-29T00:00:00Z,"{\title\"":\""Auxiliary Finance and Investment S...","\""code\"":\""641\""}"""
5,transaction,08e6f7c9-f359-48f2-b086-a55f468f2f60,posted,TFR From Transaction to CC Acc 13,-9406.0,070c1d68-0ee0-477a-9679-294ea7059939,26367.06,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,28/07/2023,2023-07-28T00:00:00Z,"{\title\"":\""Civic","Professional and Other Interest Group Services\"""
6,transaction,53570fde-c731-4e33-8224-0583650eeaa7,posted,Payroll WFRMS 15439393,17098.0,070c1d68-0ee0-477a-9679-294ea7059939,35773.06,credit,transfer,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,28/07/2023,2023-07-28T00:00:00Z,"{\title\"":\""Unknown\""","\""code\"":\""0\""}"""
7,transaction,55dbdd71-9976-43ec-a724-7cf620d279d4,posted,Manly Maths Tutor Wages,201.0,070c1d68-0ee0-477a-9679-294ea7059939,18675.06,credit,transfer,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,17/07/2023,2023-07-17T00:00:00Z,"{\title\"":\""Educational Support Services\""","\""code\"":\""822\""}"""
8,transaction,17b710a5-da3d-42f8-ae31-f9d601f95c6d,posted,MANLY WEST SCHOOL,-1422.2,070c1d68-0ee0-477a-9679-294ea7059939,18474.06,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,16/07/2023,2023-07-16T00:00:00Z,"{\title\"":\""School Education\""","\""code\"":\""802\""}"""
9,transaction,d93fb495-8a7a-4997-8a16-b3581b778471,posted,MANLY WEST SCHOOL,-51.0,070c1d68-0ee0-477a-9679-294ea7059939,19896.26,debit,payment,AU00000,1bd6c08e-8457-4a74-bdd9-514df489d27e,NaN,16/07/2023,2023-07-16T00:00:00Z,"{\title\"":\""School Education\""","\""code\"":\""802\""}"""


In [5]:
targetCol = 'balance'

### Data Cleaning

###### First, we'll drop some usless columns

In [6]:
clean = raw.copy(deep=True)
clean.drop(['type', 'id', 'status', 'direction', 'connection', 'subClass', 'links', 'description', 'postDate', 'enrich', 'institution', 'account'], axis=1, inplace=True)

clean.head(10)

,amount,balance,class,transactionDate
0,-200.0,22109.56,cash-withdrawal,3/08/2023
1,-2.5,22309.56,bank-fee,3/08/2023
2,-200.0,22312.06,cash-withdrawal,3/08/2023
3,-2.5,22512.06,bank-fee,3/08/2023
4,-3852.5,22514.56,payment,29/07/2023
5,-9406.0,26367.06,payment,28/07/2023
6,17098.0,35773.06,transfer,28/07/2023
7,201.0,18675.06,transfer,17/07/2023
8,-1422.2,18474.06,payment,16/07/2023
9,-51.0,19896.26,payment,16/07/2023


###### Clean up dates into timestamps

In [7]:
datesAsTimestamp = pd.to_datetime(clean.pop('transactionDate'), format='%d/%m/%Y')
datesAsTimestamp = datesAsTimestamp.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day

clean['Day sin'] = np.sin(datesAsTimestamp * (2 * np.pi / day))
clean['Day cos'] = np.cos(datesAsTimestamp * (2 * np.pi / day))
clean['Year sin'] = np.sin(datesAsTimestamp * (2 * np.pi / year))
clean['Year cos'] = np.cos(datesAsTimestamp * (2 * np.pi / year))

###### One hot encode descriptions (to potentially indentify major hurdles to savings?)

In [8]:
encodedClasses = pd.get_dummies(clean.pop('class'))
clean = clean.join(encodedClasses)

In [9]:
clean.head(10)

,amount,balance,Day sin,Day cos,Year sin,Year cos,bank-fee,cash-withdrawal,payment,transfer
0,-200.0,22109.56,-8.175941e-12,1.0,-0.516141,-0.856504,0,1,0,0
1,-2.5,22309.56,-8.175941e-12,1.0,-0.516141,-0.856504,1,0,0,0
2,-200.0,22312.06,-8.175941e-12,1.0,-0.516141,-0.856504,0,1,0,0
3,-2.5,22512.06,-8.175941e-12,1.0,-0.516141,-0.856504,1,0,0,0
4,-3852.5,22514.56,-5.648035e-13,1.0,-0.440652,-0.897678,0,0,1,0
5,-9406.0,26367.06,-4.863342e-12,1.0,-0.425145,-0.905125,0,0,1,0
6,17098.0,35773.06,-4.863342e-12,1.0,-0.425145,-0.905125,0,0,0,1
7,201.0,18675.06,-8.491521e-12,1.0,-0.247299,-0.968939,0,0,0,1
8,-1422.2,18474.06,-1.279006e-11,1.0,-0.230595,-0.973050,0,0,1,0
9,-51.0,19896.26,-1.279006e-11,1.0,-0.230595,-0.973050,0,0,1,0


### Train/Test Split

In [10]:
trainPercent = 0.75
validationPercent = 0.15
testPercent = 0.1

In [11]:
n = len(clean)
train_df = clean[0:int(n*trainPercent)]
val_df = clean[int(n*trainPercent):int(n*(1-(validationPercent+testPercent)))]
test_df = clean[int(n*(1-(validationPercent+testPercent))):]

### Normalisation

In [12]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [13]:
#After normalising, it's clear Day cos is useless
train_df.drop(columns=['Day cos'], inplace=True)
val_df.drop(columns=['Day cos'], inplace=True)
test_df.drop(columns=['Day cos'], inplace=True)

### Model Training

In [14]:
train_df

,amount,balance,Day sin,Year sin,Year cos,bank-fee,cash-withdrawal,payment,transfer
0,-0.051394,2.081805,-0.457899,-0.788015,-1.048682,-0.335972,2.308281,-0.988487,-0.568901
1,-0.013377,2.114899,-0.457899,-0.788015,-1.048682,2.962660,-0.431217,-0.988487,-0.568901
2,-0.051394,2.115312,-0.457899,-0.788015,-1.048682,-0.335972,2.308281,-0.988487,-0.568901
3,-0.013377,2.148406,-0.457899,-0.788015,-1.048682,2.962660,-0.431217,-0.988487,-0.568901
4,-0.754472,2.148820,1.295967,-0.676693,-1.104887,-0.335972,-0.431217,1.006963,-0.568901
...,...,...,...,...,...,...,...,...,...
211,0.025795,-0.416351,0.014096,1.026170,-0.835135,-0.335972,-0.431217,-0.988487,1.749638
212,0.025795,-0.449610,1.386294,1.061060,-0.801039,-0.335972,-0.431217,-0.988487,1.749638
213,-0.109142,-0.482869,0.395764,1.078024,-0.783579,-0.335972,-0.431217,1.006963,-0.568901
214,-0.043694,-0.400135,0.777432,1.126936,-0.729616,-0.335972,-0.431217,1.006963,-0.568901


In [15]:
model = tf.keras.Sequential([
    Input(shape=(8,)),
    Dense(64, activation='relu'),
    Dropout(0.35),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.20),
    BatchNormalization(),
    Dense(1, 'linear')
])

In [16]:
learning_rate_schedule = PiecewiseConstantDecay([1000, 2000, 3000], [0.1, 0.05, 0.01, 0.005])

In [17]:
earlyStop = EarlyStopping(monitor='mean_squared_error', patience=25, restore_best_weights=True)

In [18]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'cosine_similarity'])

In [19]:
history = model.fit(train_df.drop(columns=[targetCol]),
                    train_df[targetCol],
                    epochs=150,
                    batch_size=64,
                    validation_data = (val_df.drop(columns=[targetCol]), val_df[targetCol]),
                    callbacks = [earlyStop])

Epoch 1/150
4/4 [==============================] - 7s 46ms/step - loss: 1.2916 - mean_squared_error: 1.2916 - mean_absolute_error: 0.8710 - mean_absolute_percentage_error: 409.6808 - cosine_similarity: 0.0278
Epoch 2/150
4/4 [==============================] - 0s 20ms/step - loss: 1.2743 - mean_squared_error: 1.2743 - mean_absolute_error: 0.8412 - mean_absolute_percentage_error: 499.8542 - cosine_similarity: 0.2500
Epoch 3/150
4/4 [==============================] - 0s 23ms/step - loss: 1.0426 - mean_squared_error: 1.0426 - mean_absolute_error: 0.7592 - mean_absolute_percentage_error: 391.5389 - cosine_similarity: 0.1759
Epoch 4/150
4/4 [==============================] - 0s 21ms/step - loss: 0.8498 - mean_squared_error: 0.8498 - mean_absolute_error: 0.7201 - mean_absolute_percentage_error: 360.3646 - cosine_similarity: 0.2222
Epoch 5/150
4/4 [==============================] - 0s 22ms/step - loss: 0.9171 - mean_squared_error: 0.9171 - mean_absolute_error: 0.7341 - mean_absolute_percentage

### Model Evaluation

In [22]:
model.evaluate(test_df.drop(columns=[targetCol]), test_df[targetCol])

3/3 [==============================] - 1s 9ms/step - loss: 0.4267 - mean_squared_error: 0.4267 - mean_absolute_error: 0.5706 - mean_absolute_percentage_error: 60.5052 - cosine_similarity: 0.9178


[0.426678866147995,
 0.426678866147995,
 0.5705631375312805,
 60.505184173583984,
 0.9178082346916199]